In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from utils.models import create_vae_model
from utils.losses import reconstruction_loss
from utils.callbacks import SaveDecoderOutput, SaveDecoderModel

In [ ]:
physical_gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_gpus[0], True)

In [5]:
def parse_fn(dataset, input_size=(28, 28)):
    x = tf.cast(dataset['image'], tf.float32)
    x = tf.image.resize(x, input_size)
    x = x / 255.
    return x, x

In [6]:
dataset = 'mnist'     # 'cifar10', 'fashion_mnist', 'mnist'
log_dirs = 'logs_vae'
batch_size = 16
latent_dim = 2
input_shape = (28, 28, 1)

In [7]:
# Load datasets
train_data = tfds.load(dataset, split=tfds.Split.TRAIN)
test_data = tfds.load(dataset, split=tfds.Split.TEST)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 2/4 [00:00<00:00, 19.02 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:00<00:00, 19.02 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  3.49 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.91 url/s]


Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [8]:
# Setting datasets
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
train_data = train_data.shuffle(1000)
train_data = train_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)
test_data = test_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

In [9]:
# Callbacks function
model_dir = log_dirs + '/models'
os.makedirs(model_dir, exist_ok=True)
model_tb = keras.callbacks.TensorBoard(log_dir=log_dirs)
model_sdw = SaveDecoderModel(model_dir + '/best_model.h5', monitor='val_loss')
model_testd = SaveDecoderOutput(28, log_dir=log_dirs)

In [18]:
# create vae model
vae_model = create_vae_model(input_shape, latent_dim)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_6[0][0]                   
____________________________________________________________________________________________

In [19]:
# training
optimizer = tf.keras.optimizers.RMSprop()
vae_model.compile(optimizer, loss=reconstruction_loss)
vae_model.fit(train_data, epochs=100, validation_data=test_data, callbacks=[model_tb, model_sdw, model_testd])

Epoch 1/100
   3/3750 [..............................] - ETA: 6:29 - loss: 534.1042  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0163s vs `on_train_batch_end` time: 0.0283s). Check your callbacks.


3750/3750 [==============================] - 48s 13ms/step - loss: 162.1819 - val_loss: 149.7025
Epoch 2/100
3750/3750 [==============================] - 45s 12ms/step - loss: 148.3017 - val_loss: 145.6137
Epoch 3/100
3750/3750 [==============================] - 46s 12ms/step - loss: 144.9577 - val_loss: 143.2278
Epoch 4/100
3750/3750 [==============================] - 47s 12ms/step - loss: 143.1409 - val_loss: 144.0208
Epoch 5/100
3750/3750 [==============================] - 47s 13ms/step - loss: 141.9360 - val_loss: 141.5128


Epoch 6/100
3750/3750 [==============================] - 47s 12ms/step - loss: 140.9917 - val_loss: 140.6755


Epoch 7/100
3750/3750 [==============================] - 46s 12ms/step - loss: 140.3519 - val_loss: 141.2677
Epoch 8/100
3750/3750 [==============================] - 49s 13ms/step - loss: 139.8344 - val_loss: 139.5085


Epoch 9/100
3750/3750 [==============================] - 47s 13ms/step - loss: 139.4601 - val_loss: 139.9846
Epoch 10/100
3750/3750 [==============================] - 47s 13ms/step - loss: 139.0697 - val_loss: 139.9671
Epoch 11/100
3750/3750 [==============================] - 47s 12ms/step - loss: 138.7857 - val_loss: 139.6437
Epoch 12/100
3750/3750 [==============================] - 46s 12ms/step - loss: 138.5973 - val_loss: 139.3638


...

Epoch 35/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.3523 - val_loss: 137.2695
Epoch 36/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.2714 - val_loss: 137.7235
Epoch 37/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.2229 - val_loss: 138.4014
Epoch 38/100
3750/3750 [==============================] - 47s 13ms/step - loss: 136.2967 - val_loss: 137.3796
Epoch 39/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.1421 - val_loss: 136.8842


Epoch 40/100
3750/3750 [==============================] - 47s 12ms/step - loss: 136.0721 - val_loss: 137.1657
Epoch 41/100
3750/3750 [==============================] - 47s 12ms/step - loss: 136.0456 - val_loss: 137.6602
Epoch 42/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.1859 - val_loss: 137.0575
Epoch 43/100
3750/3750 [==============================] - 47s 13ms/step - loss: 136.0480 - val_loss: 137.2758
Epoch 44/100
3750/3750 [==============================] - 48s 13ms/step - loss: 136.1249 - val_loss: 136.9432
Epoch 45/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.9007 - val_loss: 137.3047
Epoch 46/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.9378 - val_loss: 137.1097
Epoch 47/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.8708 - val_loss: 138.8237
Epoch 48/100
3750/3750 [==============================] - 47s 12ms/step - loss: 135.9045 - val_loss: 136.9406
Epoch 49/1

Epoch 52/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.7599 - val_loss: 136.5539


Epoch 53/100
3750/3750 [==============================] - 48s 13ms/step - loss: 135.7713 - val_loss: 137.2103
Epoch 54/100
3750/3750 [==============================] - 48s 13ms/step - loss: 135.6658 - val_loss: 136.7005
Epoch 55/100
3750/3750 [==============================] - 48s 13ms/step - loss: 135.6274 - val_loss: 137.2811
Epoch 56/100
3750/3750 [==============================] - 46s 12ms/step - loss: 135.6892 - val_loss: 137.7566
Epoch 57/100
3750/3750 [==============================] - 46s 12ms/step - loss: 135.7497 - val_loss: 136.2355


Epoch 58/100
3750/3750 [==============================] - 48s 13ms/step - loss: 135.7622 - val_loss: 136.8359
Epoch 59/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.7374 - val_loss: 137.0651
Epoch 60/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.7052 - val_loss: 138.2778
Epoch 61/100
3750/3750 [==============================] - 47s 12ms/step - loss: 135.7590 - val_loss: 136.5810
Epoch 62/100
3750/3750 [==============================] - 47s 13ms/step - loss: 135.6077 - val_loss: 137.0298
Epoch 63/100
3750/3750 [==============================] - 45s 12ms/step - loss: 135.6018 - val_loss: 136.8515
Epoch 64/100
3750/3750 [==============================] - 46s 12ms/step - loss: 135.7593 - val_loss: 137.4063
Epoch 65/100
3750/3750 [==============================] - 47s 12ms/step - loss: 135.7774 - val_loss: 137.3459
Epoch 66/100
3750/3750 [==============================] - 47s 12ms/step - loss: 135.7076 - val_loss: 137.5406
Epoch 67/1